In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
!pip3 install seaborn
import seaborn as sns
import random

In [ ]:
### THIS IS PATCH 10.8 Jinx itemization analysis

## From MismatchedSock's functions
def calculate_damage(ad, armor):
    return 100/(100+armor)*ad

def calculate_crit(ad, crit_bonus, armor):
    return 100/(100+armor)*ad*(1+crit_bonus)

def calculate_time_per_auto(attack_speed):
    return 1.0 / attack_speed

# Jinx base stats, assuming level 2 jinx
jinx_base_ad = 126
jinx_base_as = 0.75
jinx_base_crit_chance = 0.25
jinx_base_crit_bonus_damage = 0.5
jinx_base_rocket = 175

# For calculations with bramble, set to 0, otherwise set to 1
bramble = 1

# Order of builds in list: [GS+GS+GA, IE+IE+GA,LW+GS+GA, IE+GS+GA, IE+LW+GA]
# target_num is either 1st target (before passive) or 2nd target (after Jinx is excited)

def sim(armor, health, target_num):
    # Must keep running total of 5 enemies at a time, for each build
    hp_left = np.array([health, health, health, health, health])
    
    # Running time for each build, for each simulation
    total_times = 1.*np.array([0,0,0,0,0])
    
    # To keep track of whether Last Whisper has shredded armor of enemy
    lw_armor= [0, 0, 0, 0, 0] 
    if target_num == 1:
        base_as = jinx_base_as
    else:
        base_as = 1.75*jinx_base_as #assumption: Get excited works off of Jinx's base attack speed
    auto = 0
    
    while True: 
        # For crit chance of basic attack
        rand = random.random()
        # For crit chance of blaster buff shot
        rand2 = random.random()

        # GS+GS+GA Calculation
        if hp_left[0] > 0:
            ad = jinx_base_ad + 15 * 3 +  2 * hp_left[0] * 0.12
            aspeed = base_as + 2*0.15*jinx_base_as
            if (rand < jinx_base_crit_chance):
                damage = calculate_crit(ad, 0.5* bramble, armor)
            else:
                damage = calculate_damage(ad, armor)
            # Blaster proc: Assume extra blaster hit always hits current target
            if auto % 4 == 3:
                rand2 = random.random()
                if (rand2 < jinx_base_crit_chance):
                    damage += calculate_crit(ad, 0.5* bramble, armor)
                else:
                    damage += calculate_damage(ad, armor)
                
            hp_left[0] = np.max((0, hp_left[0]-damage))
            total_times[0] += calculate_time_per_auto(aspeed)

        # IE+IE+GA Calculation
        if hp_left[1] > 0:
            ad = jinx_base_ad + 15 * 3
            aspeed = base_as
            if (rand < jinx_base_crit_chance + 2 * 0.2):
                damage = calculate_crit(ad, 2.5* bramble, armor)
            else:
                damage = calculate_damage(ad, armor)
                
            # Blaster proc: Assume extra blaster hit always hits current target
            if auto % 4 == 3:
                if (rand2 < jinx_base_crit_chance + 2*0.2):
                    damage += calculate_crit(ad, 2.5* bramble, armor)
                else:
                    damage += calculate_damage(ad, armor)
            hp_left[1] = np.max((0, hp_left[1]-damage))
            total_times[1] += calculate_time_per_auto(aspeed)
            
        # LW+GS+GA Calculation SORRY OUT OF ORDER
        if hp_left[2] > 0:
            ad = jinx_base_ad + 15 * 2 + hp_left[2] * 0.12
            aspeed = base_as+ 2 * 0.15 * jinx_base_as
            if (rand < jinx_base_crit_chance + 0.2):
                if lw_armor[2] > 0:
                    damage = calculate_crit(ad, 0.5* bramble, 0.1*armor)
                else:
                    damage = calculate_crit(ad, 0.5* bramble, armor)
                    lw_armor[2] = 4
            else:
                if lw_armor[2] > 0:
                    damage = calculate_damage(ad, 0.1*armor)
                else:
                    damage = calculate_damage(ad, armor)
                
            # Blaster proc: Assume extra blaster hit always hits current target
            if auto % 4 == 3:
                if (rand2 < jinx_base_crit_chance + 0.2):
                    if lw_armor[2] > 0:
                        damage += calculate_crit(ad, 0.5* bramble, 0.1*armor)
                    else:
                        damage += calculate_crit(ad, 0.5* bramble, armor)
                        lw_armor[2] = 4
                else:
                    if lw_armor[2] > 0:
                        damage += calculate_damage(ad, 0.1*armor)
                    else:
                        damage += calculate_damage(ad, armor)
            hp_left[2] = np.max((0, hp_left[2]-damage))
            total_times[2] += calculate_time_per_auto(aspeed)
            
        # IE+GS+GA Calculation
        if hp_left[3] > 0:
            ad = jinx_base_ad + 15 * 3 +  hp_left[3] * 0.12
            aspeed = base_as + 1*0.15*jinx_base_as
            if (rand < jinx_base_crit_chance + 0.2):
                damage = calculate_crit(ad, 1.5* bramble, armor)
            else:
                damage = calculate_damage(ad, armor)
            # Blaster proc: Assume extra blaster hit always hits current target
            if auto % 4 == 3:
                if (rand2 < jinx_base_crit_chance + 0.2):
                    damage += calculate_crit(ad, 1.5* bramble, armor)
                else:
                    damage += calculate_damage(ad, armor)
                
            hp_left[3] = np.max((0, hp_left[3]-damage))
            total_times[3] += calculate_time_per_auto(aspeed)
            
        # IE+LW+GA Calculation
        if hp_left[4] > 0:
            ad = jinx_base_ad + 15 * 2
            aspeed = base_as+ 0.15 * jinx_base_as
            if (rand < jinx_base_crit_chance + 2 * 0.2):
                if lw_armor[4] > 0:
                    damage = calculate_crit(ad, 1.5* bramble, 0.1*armor)
                else:
                    damage = calculate_crit(ad, 1.5* bramble, armor)
                    lw_armor[4] = 4
            else:
                if lw_armor[4] > 0:
                    damage = calculate_damage(ad, 0.1*armor)
                else:
                    damage = calculate_damage(ad, armor)
                
            # Blaster proc: Assume extra blaster hit always hits current target
            if auto % 4 == 3:
                if (rand2 < jinx_base_crit_chance + 2 * 0.2):
                    if lw_armor[4] > 0:
                        damage += calculate_crit(ad, 1.5* bramble, 0.1*armor)
                    else:
                        damage += calculate_crit(ad, 1.5* bramble, armor)
                        lw_armor[4] = 4
                else:
                    if lw_armor[4] > 0:
                        damage += calculate_damage(ad, 0.1*armor)
                    else:
                        damage += calculate_damage(ad, armor)
            hp_left[4] = np.max((0, hp_left[4]-damage))
            total_times[4] += calculate_time_per_auto(aspeed)
        
        # End current simulation once every enemy has died for all builds
        if sum(hp_left) == 0:
            break
        # Blaster counter
        auto += 1
    
    # Return time it took for each build to take down the enemy
    return total_times   
    
    

In [ ]:
###########################################################
#
# USER_INPUT 
# min and max health, min and max armor
# target = 0 for before passive, target = 1 for after passive
# num_simulations for number of trials of EACH build of EACH enemy (armor, health) value
#
# ASSUMPTIONS
# LW reduces armor for the rest of fight
#
############################################################
target = 0
min_armor = 0
max_armor = 360
min_health = 500
max_health = 2500
num_simulations = 1000

armor_increments = (max_armor - min_armor)/ 18
health_increments = (max_health - min_health)/ 20
armor_range = np.arange(min_armor, max_armor + 1, armor_increments)
health_range = np.arange(min_health, max_health + 1, health_increments)

data = np.zeros((len(armor_range), len(health_range)))

for i, armor in enumerate(armor_range):
    for j, health in enumerate(health_range):
        totals = np.zeros(5)
        for k in range(0, num_simulations):
            res = sim(armor, health, target)
            totals += res
        totals /= num_simulations
        # Keep track of best build for armor/health combo
        data[i][j] = np.argmin(totals)
        
# Convert data for plot
data_df = pd.DataFrame(data, columns=health_range.astype(int), index=armor_range.astype(int))
data_df = data_df.iloc[:, ::-1]


fig = plt.figure(figsize=(15, 15))
plt.rcParams["axes.labelsize"] = 30
n = 5   
# Generate Discrete colormap (n samples from a given cmap)
cmap = sns.color_palette("Set2", n, desat=1)
ax = sns.heatmap(data_df.T, cmap=cmap)
colorbar = ax.collections[0].colorbar 
r = colorbar.vmax - colorbar.vmin 
colorbar.set_ticks([colorbar.vmin + r / n * (0.5 + i) for i in range(n)])
colorbar.set_ticklabels(['GS/GS/GA', 'IE/IE/GA','LW/GS/GA', 'IE/GS/GA', 'IE/LW/GA'])  
colorbar.ax.tick_params(labelsize=20) 
ax.set_title('Optimal Jinx itemization, before passive, with bramble', fontsize=25)
for label in ax.xaxis.get_ticklabels()[::2]:
    label.set_visible(False)
for label in ax.yaxis.get_ticklabels()[::2]:
    label.set_visible(False)
ax.tick_params(labelsize=20)
fig.patch.set_facecolor((1, 1, 1))
plt.xlabel('Armor')
plt.ylabel('Health')
plt.show()